# ELGC-Net 快速训练 (GitHub集成版)

这个Notebook直接使用GitHub中的配置文件，无需手动创建任何文件。

## 步骤概览
1. 挂载Google Drive
2. 克隆项目
3. 安装依赖
4. 准备数据集
5. 开始训练
6. 评估模型（可选）
7. 保存结果（可选）

In [1]:
# 1. 挂载Google Drive并设置环境
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# 检查GPU
!nvidia-smi

Mounted at /content/drive
Fri Jun  6 08:48:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+---------------------

In [2]:
# 2. 克隆项目（包含colab文件夹）
!rm -rf /content/elgcnet  # 清理旧文件
!git clone https://github.com/nihaofana/elgcnet.git
%cd /content/elgcnet

# 查看colab目录
!ls -la colab/

Cloning into 'elgcnet'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 153 (delta 46), reused 53 (delta 24), pack-reused 73 (from 2)
Receiving objects: 100% (153/153), 2.10 MiB | 30.25 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/elgcnet
total 56
drwxr-xr-x 2 root root 4096 Jun  6 08:48 .
drwxr-xr-x 8 root root 4096 Jun  6 08:48 ..
-rw-r--r-- 1 root root  560 Jun  6 08:48 colab_config.py
-rw-r--r-- 1 root root 4694 Jun  6 08:48 correct_notebook.ipynb
-rw-r--r-- 1 root root 2311 Jun  6 08:48 eval_colab.py
-rw-r--r-- 1 root root 2122 Jun  6 08:48 monitor_training.py
-rw-r--r-- 1 root root 6096 Jun  6 08:48 prepare_data_balanced.py
-rw-r--r-- 1 root root 3046 Jun  6 08:48 prepare_data.py
-rw-r--r-- 1 root root 1650 Jun  6 08:48 README.md
-rw-r--r-- 1 root root 1911 Jun  6 08:48 save_results.py
-rw-r--r-- 1 root root 2755 Jun  6 08:48 train_colab.py
-rw-r--r-- 1 root root 

In [3]:
# 3. 安装依赖
!pip install einops==0.3.2 timm==0.9.16 fvcore opencv-python scikit-learn tqdm -q

import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 600.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# 4. 准备数据集（仅复制部分数据以加快训练）
# !python colab/prepare_data.py --train_samples 100 --val_samples 25 --test_samples 25

In [5]:
# # 删除之前下载的数据集
# !rm -rf ./datasets/CD/LEVIR-CD-256

# # 确认删除完成
# !ls -la ./datasets/CD/

In [ ]:
# 使用改进脚本，大幅增加数据量
!python colab/prepare_data_balanced.py --train_samples 1000 --val_samples 200 --test_samples 200

开始智能选择数据集样本...

分析 train 集...
  分析进度: 0/7120
  分析进度: 100/7120
  分析进度: 200/7120
  分析进度: 300/7120
  分析进度: 400/7120
  分析进度: 500/7120
  分析进度: 600/7120
  分析进度: 700/7120
  分析进度: 800/7120
  分析进度: 900/7120


In [ ]:
# 5. 开始训练
!rm -rf ./checkpoints/elgcnet_levir_colab
!python colab/train_colab.py

## 可选操作

In [ ]:
# 6. 评估模型（可选）
!python colab/eval_colab.py

In [ ]:
!python analyze_dataset.py

In [ ]:
# 直接使用加权损失重新训练，不需要重新准备数据
!python colab/train_colab_weighted.py

In [ ]:
# 创建上面的 eval_weighted.py 文件，然后运行
!python eval_weighted.py

In [ ]:
# 先检查数组的实际结构
import numpy as np

try:
    # 加载数据
    weighted_train = np.load('./checkpoints/elgcnet_levir_colab_weighted/train_acc.npy')
    weighted_val = np.load('./checkpoints/elgcnet_levir_colab_weighted/val_acc.npy')

    print("📊 数据结构分析:")
    print(f"训练数据形状: {weighted_train.shape}")
    print(f"验证数据形状: {weighted_val.shape}")
    print(f"训练数据类型: {type(weighted_train)}")
    print(f"验证数据类型: {type(weighted_val)}")

    print(f"\n训练数据内容:")
    print(weighted_train)

    print(f"\n验证数据内容:")
    print(weighted_val)

    # 如果是1D数组，可能只保存了最佳分数
    if weighted_train.ndim == 1:
        print(f"\n⚠️ 注意: 数据是1D格式")
        print(f"训练数据长度: {len(weighted_train)}")
        print(f"验证数据长度: {len(weighted_val)}")

except Exception as e:
    print(f"❌ 无法加载数据: {e}")

In [ ]:
# 修复版：正确处理训练历史数据
import numpy as np
import matplotlib.pyplot as plt
import os

def analyze_weighted_training_history():
    """正确分析加权训练历史"""

    checkpoint_dir = './checkpoints/elgcnet_levir_colab_weighted'

    try:
        # 检查并加载训练历史
        train_file = os.path.join(checkpoint_dir, 'train_acc.npy')
        val_file = os.path.join(checkpoint_dir, 'val_acc.npy')

        if os.path.exists(train_file) and os.path.exists(val_file):
            weighted_train = np.load(train_file)
            weighted_val = np.load(val_file)

            print("📊 加权训练历史分析:")
            print(f"训练数据形状: {weighted_train.shape}")
            print(f"验证数据形状: {weighted_val.shape}")

            # 根据实际格式处理数据
            if weighted_train.ndim == 1:
                # 1D格式 - 可能每个元素代表一个epoch的分数
                print(f"训练轮数: {len(weighted_train)}")
                print(f"训练分数变化: {weighted_train}")
                print(f"验证分数变化: {weighted_val}")
                print(f"最佳训练分数: {np.max(weighted_train):.4f}")
                print(f"最佳验证分数: {np.max(weighted_val):.4f}")
                print(f"最终训练分数: {weighted_train[-1]:.4f}")
                print(f"最终验证分数: {weighted_val[-1]:.4f}")

                # 绘制1D数据
                plt.figure(figsize=(10, 6))
                epochs = range(len(weighted_train))
                plt.plot(epochs, weighted_train, 'b-o', label='Train Score', linewidth=2)
                plt.plot(epochs, weighted_val, 'r-o', label='Val Score', linewidth=2)
                plt.title('加权训练历史')
                plt.xlabel('Epoch')
                plt.ylabel('Score')
                plt.legend()
                plt.grid(True, alpha=0.3)
                plt.show()

            elif weighted_train.ndim == 2:
                # 2D格式 - 每行包含多个指标
                print(f"训练轮数: {weighted_train.shape[0]}")
                print(f"指标数量: {weighted_train.shape[1]}")

                # 假设列顺序是 [acc, miou, mf1, ...]
                if weighted_train.shape[1] >= 3:
                    print(f"最佳训练mF1: {np.max(weighted_train[:, 2]):.4f}")
                    print(f"最佳验证mF1: {np.max(weighted_val[:, 2]):.4f}")
                    print(f"最终训练mF1: {weighted_train[-1, 2]:.4f}")
                    print(f"最终验证mF1: {weighted_val[-1, 2]:.4f}")

                    # 绘制2D数据
                    plt.figure(figsize=(15, 5))

                    plt.subplot(1, 3, 1)
                    plt.plot(weighted_train[:, 0], 'b-', label='Train Acc')
                    plt.plot(weighted_val[:, 0], 'r-', label='Val Acc')
                    plt.title('Accuracy')
                    plt.legend()
                    plt.grid(True)

                    plt.subplot(1, 3, 2)
                    plt.plot(weighted_train[:, 1], 'b-', label='Train mIoU')
                    plt.plot(weighted_val[:, 1], 'r-', label='Val mIoU')
                    plt.title('mIoU')
                    plt.legend()
                    plt.grid(True)

                    plt.subplot(1, 3, 3)
                    plt.plot(weighted_train[:, 2], 'b-', label='Train mF1')
                    plt.plot(weighted_val[:, 2], 'r-', label='Val mF1')
                    plt.title('mF1')
                    plt.legend()
                    plt.grid(True)

                    plt.tight_layout()
                    plt.show()

        else:
            print("❌ 训练历史文件不存在")

    except Exception as e:
        print(f"❌ 分析训练历史时出错: {e}")
        import traceback
        traceback.print_exc()

# 同时检查日志文件
def check_training_log():
    """检查训练日志文件"""
    log_file = './checkpoints/elgcnet_levir_colab_weighted/log.txt'

    if os.path.exists(log_file):
        print("\n📄 训练日志分析:")
        with open(log_file, 'r') as f:
            lines = f.readlines()

        # 查找关键信息
        best_epochs = []
        final_metrics = []

        for line in lines:
            if "Best model updated" in line:
                best_epochs.append(line.strip())
            if "epoch_mF1=" in line and "Is_training: False" in line:
                final_metrics.append(line.strip())

        print(f"最佳模型更新次数: {len(best_epochs)}")
        if best_epochs:
            print("最佳模型更新记录:")
            for update in best_epochs[-3:]:  # 显示最后3次更新
                print(f"  {update}")

        if final_metrics:
            print(f"\n最后几次验证结果:")
            for metric in final_metrics[-3:]:  # 显示最后3次验证
                print(f"  {metric}")
    else:
        print("❌ 日志文件不存在")

# 运行分析
analyze_weighted_training_history()
check_training_log()

In [ ]:
# 7. 保存结果到Drive（可选）
!python colab/save_results.py

In [ ]:
# 8. 监控GPU使用（可选）
# 在训练过程中，在新的cell中运行此代码
import sys
sys.path.append('/content/elgcnet/colab')
from monitor_training import monitor_gpu

# 监控60秒
monitor_gpu(duration=60, interval=2)

## 配置说明

### 修改训练参数
如需修改训练参数，编辑 `colab/colab_config.py`：

```python
class ColabConfig:
    BATCH_SIZE = 8      # 批次大小
    MAX_EPOCHS = 100    # 训练轮数
    LEARNING_RATE = 0.00031  # 学习率
```

### 数据集路径
确保数据集在Google Drive的以下位置：
```
/content/drive/My Drive/Change_Detection/LEVIR-CD-256/
```

### 故障排除
1. **GPU内存不足**：减小BATCH_SIZE
2. **找不到数据集**：检查Google Drive路径
3. **训练中断**：模型会自动保存，可以从检查点恢复